In [1]:
%pip install transformers
%pip install torch
%pip install typing
%pip install re

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List
import torch
import re

class SentimentModel():
    def __init__(self, model_name: str = "oliverguhr/german-sentiment-bert"):
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'        
            
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model = self.model.to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.clean_chars = re.compile(r'[^A-Za-züöäÖÜÄß ]', re.MULTILINE)
        self.clean_http_urls = re.compile(r'https*\S+', re.MULTILINE)
        self.clean_at_mentions = re.compile(r'@\S+', re.MULTILINE)

    def predict_sentiment(self, texts: List[str], output_probabilities = False)-> List[str]:
        texts = [self.clean_text(text) for text in texts]
        # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        # truncation=True limits number of tokens to model's limitations (512)
        encoded = self.tokenizer.batch_encode_plus(texts,padding=True, add_special_tokens=True,truncation=True, return_tensors="pt")
        encoded = encoded.to(self.device)
        with torch.no_grad():
                logits = self.model(**encoded)
        
        label_ids = torch.argmax(logits[0], axis=1)

        if output_probabilities == False:
            return [self.model.config.id2label[label_id.item()] for label_id in label_ids]
        else:
            predictions = torch.softmax(logits[0], dim=-1).tolist()  
            probabilities = []
            for prediction in predictions:
                probabilities += [[[self.model.config.id2label[index], item] for index, item in enumerate(prediction)]]
                
            return [self.model.config.id2label[label_id.item()] for label_id in label_ids], probabilities
        
        
    def replace_numbers(self,text: str) -> str:
        return text.replace("0"," null").replace("1"," eins").replace("2"," zwei")\
            .replace("3"," drei").replace("4"," vier").replace("5"," fünf") \
            .replace("6"," sechs").replace("7"," sieben").replace("8"," acht") \
            .replace("9"," neun")         

    def clean_text(self,text: str)-> str:    
            text = text.replace("\n", " ")        
            text = self.clean_http_urls.sub('',text)
            text = self.clean_at_mentions.sub('',text)        
            text = self.replace_numbers(text)                
            text = self.clean_chars.sub('', text) # use only text chars                          
            text = ' '.join(text.split()) # substitute multiple whitespace with single whitespace   
            text = text.strip().lower()
            return text

c:\Users\Martin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from influx_io import load_unlabeled_reddit_posts_from_influx
import math

# Your class (keep as-is) + create model
model = SentimentModel("mdraw/german-news-sentiment-bert")

def _pick_conf(prob_list, chosen_label: str) -> float:
    """
    prob_list example: [["negative", 0.12], ["neutral", 0.70], ["positive", 0.18]]
    """
    for lbl, p in prob_list:
        if str(lbl).lower() == str(chosen_label).lower():
            return float(p)
    return 0.0

def label_unlabeled_reddit_posts(
    lookback: str = "30d",
    limit: int = 500,
    batch_size: int = 32,
):
    df = load_unlabeled_reddit_posts_from_influx(lookback=lookback, limit=limit)
    if df.empty:
        print("No unlabeled reddit_post points found.")
        return 0

    # Build the text you want to classify (title + selftext)
    texts = []
    for _, row in df.iterrows():
        title = str(row.get("title") or "").strip()
        body = str(row.get("selftext") or "").strip()
        combined = (title + " " + body).strip()
        texts.append(combined if combined else title)

    updates = []
    total = len(df)

    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)
        batch_texts = texts[start:end]

        labels, probs = model.predict_sentiment(batch_texts, output_probabilities=True)

        for i in range(start, end):
            chosen_label = labels[i - start]
            chosen_prob = _pick_conf(probs[i - start], chosen_label)

            r = df.iloc[i]
            updates.append({
                "usid": r["usid"],
                "source": r["source"],
                "_time": r["_time"],          # IMPORTANT: same timestamp to update the point
                "stance_label": chosen_label, # store model label
                "stance_conf": chosen_prob,   # store confidence
            })

    written = write_reddit_stance_updates(updates)
    print(f"Labeled+updated {written} points.")
    return written

# Run it:
label_unlabeled_reddit_posts(lookback="30d", limit=2000, batch_size=32)


NameError: name 'load_unlabeled_reddit_posts_from_influx' is not defined